In [ ]:
# Load Required Packages
import sklearn
import statistics
import scipy as sc
import numpy as np
import pandas as pd
import random
import cvxpy as cp
import matplotlib.pyplot as plt
import multiprocessing as mp
from multiprocessing import Pool
import time
import re
from sklearn import preprocessing
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import matplotlib
import mosek
import statsmodels.api as sm
import gc
import pickle
import seaborn as sns
from pmlb import fetch_data
from pmlb import classification_dataset_names, regression_dataset_names
import matplotlib
new_rc_params = {'text.usetex': True,
         'svg.fonttype': 'none',
         'text.latex.preamble': r'\usepackage{libertine}',
         'font.size': 20,
         'font.family': 'Linux Libertine',
         'mathtext.fontset': 'custom',
         'mathtext.rm': 'libertine',
         'mathtext.it': 'libertine:italic',
         'mathtext.bf': 'libertine:bold'
         }
matplotlib.rcParams.update(new_rc_params)

In [ ]:
cleve = pd.read_csv('../bagboostresults/cleve_bagboost_5fold.csv')
cleve.drop(0,axis = 0,inplace = True)
cleve.columns = ['nonzero','polish_mean','polish_std','base_mean','base_std','full_mean','full_std']
cleve = cleve.astype(float)
cleve = cleve.append({'nonzero':0, 'polish_mean':0.5, 'polish_std':0, 'base_mean':0.5, 'base_std':0, 'full_mean':cleve['full_mean'].values[0], 'full_std': cleve['full_std'].values[0]},ignore_index=True)
cleve = cleve.astype(float) 
cleve = cleve.sort_values('nonzero',ascending = True)

adult = pd.read_csv('../bagboostresults/adult_bagboost_5fold.csv')
adult.drop(0,axis = 0,inplace = True)
adult.columns = ['nonzero','polish_mean','polish_std','base_mean','base_std','full_mean','full_std']
adult = adult.astype(float)
adult = adult.append({'nonzero':0, 'polish_mean':0.5, 'polish_std':0, 'base_mean':0.5, 'base_std':0, 'full_mean':adult['full_mean'].values[0], 'full_std': adult['full_std'].values[0]},ignore_index=True)
adult = adult.astype(float)  
adult = adult.sort_values('nonzero',ascending = True)

audit = pd.read_csv('../bagboostresults/audit_bagboost_10fold.csv')
audit.drop(0,axis = 0,inplace = True)
audit.columns = ['nonzero','polish_mean','polish_std','base_mean','base_std','full_mean','full_std']
audit = audit.astype(float)
audit = audit.append({'nonzero':0, 'polish_mean':0.5, 'polish_std':0, 'base_mean':0.5, 'base_std':0, 'full_mean':audit['full_mean'].values[0], 'full_std': audit['full_std'].values[0]},ignore_index=True)
audit = audit.astype(float)   
audit = audit.sort_values('nonzero',ascending = True)

twonorm = pd.read_csv('../bagboostresults/twonorm_bagboost_5fold.csv')
twonorm.drop(0,axis = 0,inplace = True)
twonorm.columns = ['nonzero','polish_mean','polish_std','base_mean','base_std','full_mean','full_std']
twonorm = twonorm.astype(float)
twonorm = twonorm.append({'nonzero':0, 'polish_mean':0.5, 'polish_std':0, 'base_mean':0.5, 'base_std':0, 'full_mean':twonorm['full_mean'].values[0], 'full_std': twonorm['full_std'].values[0]},ignore_index=True)
twonorm = twonorm.astype(float)    
twonorm = twonorm.sort_values('nonzero',ascending = True)

gene = pd.read_csv('../bagboostresults/genewest_bagboost_5fold.csv')
gene.drop(0,axis = 0,inplace = True)
gene.columns = ['nonzero','polish_mean','polish_std','base_mean','base_std','full_mean','full_std']
gene = gene.astype(float)
gene = gene.append({'nonzero':0, 'polish_mean':0.5, 'polish_std':0, 'base_mean':0.5, 'base_std':0, 'full_mean':twonorm['full_mean'].values[0], 'full_std': twonorm['full_std'].values[0]},ignore_index=True)
gene = gene.astype(float)    
gene = gene.sort_values('nonzero',ascending = True)

In [ ]:
gene

In [ ]:
def plot_controlburn_baseline(agg,ax,title,ylabel = 'ROC AUC',uselegend = False):
    ax.plot(agg['nonzero'],agg['polish_mean'],color = 'orangered',label =  r'\textsc{ControlBurn}',linestyle = '--',linewidth =3)
    
    
    
    ax.fill_between(agg['nonzero'], agg['polish_mean']-agg['polish_std'],
                agg['polish_mean']+agg['polish_std'],facecolor='orangered',alpha=0.2)
    
    ax.plot(agg['nonzero'],agg['base_mean'],label = 'Random Forest Baseline',color = 'darkseagreen',linestyle = 'dotted',linewidth =3)
    
    ax.scatter(agg['nonzero'],agg['base_mean'],color = 'darkseagreen')
    ax.scatter(agg['nonzero'],agg['polish_mean'],color = 'orangered')
    
    ax.fill_between(agg['nonzero'], agg['base_mean'] - agg['base_std'],
                 agg['base_mean'] + agg['base_std'],facecolor='darkseagreen',alpha=0.2)
    
    ax.axhline(agg['full_mean'].values[2],color = 'grey', label = 'Random Forest Full Model')
    if uselegend:
        ax.legend()
    ax.set_xlim([0,10])
    ax.set_title(title)
    ax.set_ylabel(ylabel,labelpad = 10)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
fig = plt.figure(figsize=(18,16))

gs = gridspec.GridSpec(3, 6,figure = fig)
ax1a = plt.subplot(gs[0, 0:2])
ax1b = plt.subplot(gs[0, 2:4])
ax2a = plt.subplot(gs[0, 4:])

ax2b = plt.subplot(gs[1, 1:3])
ax2c = plt.subplot(gs[1, 3:5])
axes = [ax1a,ax1b,ax2a,ax2b,ax2c]

plot_controlburn_baseline(agg = cleve,ax = ax1a,title = 'Cleveland Heart Disease (PMLB)')
plot_controlburn_baseline(agg = adult,ax = ax2a,title = 'Adult Income (Case Study)')
plot_controlburn_baseline(agg = audit,ax = ax2b,title = 'Audit (Case Study)')
plot_controlburn_baseline(agg = twonorm,ax = ax1b,title = 'TwoNorm (PMLB)',uselegend = True)
plot_controlburn_baseline(agg = gene,ax = ax2c,title = 'Gene Expression (Case Study)',ylabel = 'Accuracy')

fig.tight_layout()
fig.text(0.5, 0.3, 'Number of Features Selected', ha='center', va='center')

plt.savefig('figures/grid_lineplot_bagboost.pdf',bbox_inches = 'tight', pad_inches = 0)